Final Tutorial

In [48]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import spotipy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Use xml files to scrape from spotify charts
for each track show their streams per day, maybe as an array or another dataframe
from there spotipy api can be used to observe the details of these songs

In [49]:
# Collect links from spotify charts top 200 streams per day
ref_str = "https://spotifycharts.com/regional/global/daily/"
ref_arr = []

# gets every day from janurary 2017 to October 2020

# for year in range(2017, 2021):
for year in range(2017, 2018):
    date = ""
    
    endingMonth = 12
    if year == 2020:
        endingMonth = 10
        
    # for month in range (1, endingMonth + 1):
    for month in range (1,2):
       
        dayCount = -1

        #gets proper day count per month
        thirtyDayCountMonths = [4, 6, 9, 11]
        if month == 2:
            dayCount = 29
        elif month in thirtyDayCountMonths:
            dayCount = 30
        else:
            dayCount = 31

        if int(month) < 10:
            month = "0" + str(month)
        for day in range (1, 16):
           
            if int(day) < 10:
                day = "0" + str(day)

            date = str(year) + "-" + str(month) + "-" + str(day) + "/download"
            date = ref_str + date
            ref_arr.append(date)

ref_arr

['https://spotifycharts.com/regional/global/daily/2017-01-01/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-02/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-03/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-04/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-05/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-06/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-07/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-08/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-09/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-10/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-11/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-12/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-13/download',
 'https://spotifycharts.com/regional/global/daily/2017-01-14/dow

In [50]:
#Loop downloading and appending of dataframes 

df = pd.DataFrame(columns =['position', 'track_name', 'artist', 'streams', 'url', 'date'] )
#make dir to save to
path = "sheets"
try:
    os.mkdir(path)
except OSError:
    print ("Folder already exists")

for i in ref_arr:
    r = requests.get(i, allow_redirects = True)
    #String manipulation to read from the correct csv files
    date = i[48:58]
    #print(date)
    fileName = "regional-global-daily-" + date + ".csv"
    #print(fileName)
    open(fileName, "wb").write(r.content)

    #os.rename(fileName, "sheets/" + fileName)

    df_new = pd.read_csv(fileName)
    df_new.columns= ['position', 'track_name', 'artist', 'streams', 'url']
    df_new['date'] = date
    
    df_new = df_new.iloc[1:] #deletes junk row from csv conversion
    df = df.append(df_new)



df.drop(['position'], axis=1, inplace=True) #delete position row since rank alraedy has this information

Folder already exists


In [51]:
df

,track_name,artist,streams,url,date
1,Starboy,The Weeknd,3135625,https://open.spotify.com/track/5aAx2yezTd8zXrk...,2017-01-01
2,Closer,The Chainsmokers,3015525,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,2017-01-01
3,Let Me Love You,DJ Snake,2545384,https://open.spotify.com/track/4pdPtRcBmOSQDlJ...,2017-01-01
4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,https://open.spotify.com/track/5knuzwU65gJK7IF...,2017-01-01
5,One Dance,Drake,2259887,https://open.spotify.com/track/1xznGGDReH1oQq0...,2017-01-01
...,...,...,...,...,...
196,You & Me,Marc E. Bassy,348357,https://open.spotify.com/track/4QdovcM8j50QPly...,2017-01-15
197,Can't Feel My Face,The Weeknd,347804,https://open.spotify.com/track/6RsWqX8zABZLhZy...,2017-01-15
198,White Iverson,Post Malone,342787,https://open.spotify.com/track/6eT7xZZlB2mwyzJ...,2017-01-15
199,Te Quiero Pa´Mi,Don Omar,342480,https://open.spotify.com/track/3BY2mafsbsoKGqS...,2017-01-15


In [52]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials


SPOTIPY_CLIENT_ID="ea1a162fbc6f413990542b76ab82a168"
SPOTIPY_CLIENT_SECRET="a09882042ce54f158fdd2b6baaf2b26d"
SPOTIPY_CLIENT_REDIRECT="http://www.cs.umd.edu/class/fall2020/cmsc320-0201/"

scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET, redirect_uri=SPOTIPY_CLIENT_REDIRECT))

results = sp.current_user_saved_tracks()


# How to get audio features of a track from our data frame
#not needed anymore but good refrence for how to get trackdata
#trackName = df.iloc[0].at['track_name']
#trackNameWithoutSpaces = '+'.join(trackName.split())
#print(trackNameWithoutSpaces)
#trackItem = sp.search(trackNameWithoutSpaces, 1, 0)
#track_id = trackItem['tracks']['items'][0]['id']
#audiofeatures = sp.audio_features(track_id)
#print(track_id)
#audiofeatures[0]

In [ ]:
#gets artist genres and artist ids for each artist in dataframe and puts them in arrays
artist_genres = []
artist_ids = []

for index, row in df.iterrows():
    artist = row['artist']
    #print(index)
    #print(artist)
    trackArtistWithoutSpaces = '+'.join(artist.split())
    result = sp.search(trackArtistWithoutSpaces)
    track = result['tracks']['items'][0]
    artist_id = track["artists"][0]["id"]
    #print(artist_id)
    #print(track)
    artist_ids.append(artist_id)
    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    artist_genres.append(artist["genres"])
    #print(artist["genres"])



In [ ]:
#add artist id and genres to dataframe
df['Artist Id'] = artist_ids
df['artist_genres'] = artist_genres

In [ ]:
#print(df["artist_genres"])

#creates new column in dataframe based on genre filter within filter func
def filt_func(genre_list):
    genre = ['pop','rap','edm','rock','indie']
    result = list(filter(lambda x: x in genre, genre_list))
    return "other" if len(result) == 0 else result[0]
    #print(result)
df['genre'] = df['artist_genres'].apply(lambda x: filt_func(x))
df['streams'] = df['streams'].apply(lambda x: int(x))

In [29]:
audioFeaturesDf = pd.DataFrame(columns=["duration_ms", "key", "mode", "time_signature", "acousticness", "danceability", "energy", "instrumentalness", "liveness", "loudness", "speechiness", "valence", "tempo", "id", "uri", "track_href", "analysis_url", "type"])
#for index, row in df.iterrows():
trackName = df.iloc[0].at['track_name']
trackNameWithoutSpaces = '+'.join(trackName.split())
trackItem = sp.search(trackNameWithoutSpaces, 1, 0)
track_id = trackItem['tracks']['items'][0]['id']
audiofeatures = sp.audio_features(track_id)
print(track_id)
print(audiofeatures[0]['danceability'])
print(row['track_name'])
    

7MXVkk9YMctZqd1Srtv4MB
0.679


NameError: name 'row' is not defined

In [53]:
trackName = df.iloc[0].at['track_name']
trackNameWithoutSpaces = '+'.join(trackName.split())
searchQuery = sp.search(trackNameWithoutSpaces, 1, 0)
track_object = searchQuery['tracks']['items'][0]
artist_object = track_object['artists'][0]
artist_id = artist_object['id']
track_id = track_object['id']
audiofeatures = sp.audio_features(track_id)
print(track_id)
print(artist_id)
print(audiofeatures[0]['danceability'])
print(trackNameWithoutSpaces)

7MXVkk9YMctZqd1Srtv4MB
1Xyo4u8uXC1ZmMpatF05PJ
0.679
Starboy


In [77]:
artist_id_list = []
track_id_list = []
audioFeaturesDf = pd.DataFrame()
# This gets the artist id and the track id (which is included in the "audio_features" search we make)
# Does this by making one search through the api and gets the ids from the information returned
for index, row in df.iterrows():
    trackName = df.iloc[index - 1].at['track_name']
    trackNameWithoutSpaces = '+'.join(trackName.split())
    searchQuery = sp.search(trackNameWithoutSpaces, 1, 0)
    track_id = ""
    artist_id = ""
    if (len(searchQuery['tracks']['items']) != 0):
        track_object = searchQuery['tracks']['items'][0]
        artist_object = track_object['artists'][0] if type(track_object['artists']) is list else track_object['artists']
        artist_id = artist_object['id']
        track_id = track_object['id']
        artist_id_list.append(artist_id)
        track_id_list.append(track_id)
    #print(trackItem)
    else:
        # If our query returned nothing then append a nan in the place of artist and track for this entry
        artist_id_list.append(np.nan)
        track_id_list.append(np.nan)
    audiofeatures = {'duration_ms' : np.nan, 'key' : np.nan, 'mode' : np.nan, 'time_signature' : np.nan, 'acousticness' : np.nan, 'danceability' : np.nan, 'energy' : np.nan, 'instrumentalness' : np.nan, 'liveness' : np.nan, 'loudness' : np.nan, 'speechiness' : np.nan, 'valence' : np.nan, 'tempo' : np.nan, 'id' : np.nan, 'uri' : np.nan, 'track_href' : np.nan, 'analysis_url' : np.nan, 'type' : np.nan, }
    #print(audiofeatures)
    # If we successfully found a track when we did our seach, then get the audio features for that
    if (track_id != ""):
        audiofeatures = sp.audio_features(track_id)[0]
    #print(track_id)
    #print(audiofeatures)
    audioFeaturesDf = audioFeaturesDf.append(audiofeatures, ignore_index=True)
    
audioFeaturesDf

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.14100,https://api.spotify.com/v1/audio-analysis/7MXV...,0.679,230453.0,0.587,7MXVkk9YMctZqd1Srtv4MB,0.000006,7.0,0.1370,-7.015,1.0,0.2760,186.003,4.0,https://api.spotify.com/v1/tracks/7MXVkk9YMctZ...,audio_features,spotify:track:7MXVkk9YMctZqd1Srtv4MB,0.486
1,0.41400,https://api.spotify.com/v1/audio-analysis/7BKL...,0.748,244960.0,0.524,7BKLCZ1jbUBVqRi2FVlTVw,0.000000,8.0,0.1110,-5.599,1.0,0.0338,95.010,4.0,https://api.spotify.com/v1/tracks/7BKLCZ1jbUBV...,audio_features,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,0.661
2,0.23500,https://api.spotify.com/v1/audio-analysis/3ibK...,0.656,256733.0,0.578,3ibKnFDaa3GhpPGlOUj7ff,0.000000,7.0,0.1180,-8.970,0.0,0.0922,94.514,4.0,https://api.spotify.com/v1/tracks/3ibKnFDaa3Gh...,audio_features,spotify:track:3ibKnFDaa3GhpPGlOUj7ff,0.556
3,0.40600,https://api.spotify.com/v1/audio-analysis/5knu...,0.720,251088.0,0.763,5knuzwU65gJK7IF5yJsuaW,0.000000,9.0,0.1800,-4.068,0.0,0.0523,101.965,4.0,https://api.spotify.com/v1/tracks/5knuzwU65gJK...,audio_features,spotify:track:5knuzwU65gJK7IF5yJsuaW,0.742
4,0.00776,https://api.spotify.com/v1/audio-analysis/1zi7...,0.792,173987.0,0.625,1zi7xx7UVEFkmKfv06H8x0,0.001800,1.0,0.3290,-5.609,1.0,0.0536,103.967,4.0,https://api.spotify.com/v1/tracks/1zi7xx7UVEFk...,audio_features,spotify:track:1zi7xx7UVEFkmKfv06H8x0,0.370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.23300,https://api.spotify.com/v1/audio-analysis/6eT7...,0.675,256533.0,0.562,6eT7xZZlB2mwyzJ2sUKG6w,0.000000,7.0,0.0816,-7.678,1.0,0.0352,130.098,4.0,https://api.spotify.com/v1/tracks/6eT7xZZlB2mw...,audio_features,spotify:track:6eT7xZZlB2mwyzJ2sUKG6w,0.309
2996,0.40700,https://api.spotify.com/v1/audio-analysis/1ISs...,0.493,198253.0,0.738,1ISsiC4Fw6f96kZQegLGiJ,0.000000,5.0,0.1300,-6.921,1.0,0.2620,176.071,4.0,https://api.spotify.com/v1/tracks/1ISsiC4Fw6f9...,audio_features,spotify:track:1ISsiC4Fw6f96kZQegLGiJ,0.219
2997,0.03800,https://api.spotify.com/v1/audio-analysis/1h2x...,0.633,245493.0,0.634,1h2xVEoJORqrg71HocgqXd,0.006400,8.0,0.0385,-12.115,1.0,0.0725,100.499,4.0,https://api.spotify.com/v1/tracks/1h2xVEoJORqr...,audio_features,spotify:track:1h2xVEoJORqrg71HocgqXd,0.872
2998,0.07170,https://api.spotify.com/v1/audio-analysis/1NhP...,0.516,224693.0,0.764,1NhPKVLsHhFUHIOZ32QnS2,0.000000,2.0,0.1150,-6.223,1.0,0.0366,148.021,4.0,https://api.spotify.com/v1/tracks/1NhPKVLsHhFU...,audio_features,spotify:track:1NhPKVLsHhFUHIOZ32QnS2,0.376


In [78]:
audioFeaturesDf['artist_id'] = artist_id_list
audioFeaturesDf

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,artist_id
0,0.14100,https://api.spotify.com/v1/audio-analysis/7MXV...,0.679,230453.0,0.587,7MXVkk9YMctZqd1Srtv4MB,0.000006,7.0,0.1370,-7.015,1.0,0.2760,186.003,4.0,https://api.spotify.com/v1/tracks/7MXVkk9YMctZ...,audio_features,spotify:track:7MXVkk9YMctZqd1Srtv4MB,0.486,1Xyo4u8uXC1ZmMpatF05PJ
1,0.41400,https://api.spotify.com/v1/audio-analysis/7BKL...,0.748,244960.0,0.524,7BKLCZ1jbUBVqRi2FVlTVw,0.000000,8.0,0.1110,-5.599,1.0,0.0338,95.010,4.0,https://api.spotify.com/v1/tracks/7BKLCZ1jbUBV...,audio_features,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,0.661,69GGBxA162lTqCwzJG5jLp
2,0.23500,https://api.spotify.com/v1/audio-analysis/3ibK...,0.656,256733.0,0.578,3ibKnFDaa3GhpPGlOUj7ff,0.000000,7.0,0.1180,-8.970,0.0,0.0922,94.514,4.0,https://api.spotify.com/v1/tracks/3ibKnFDaa3Gh...,audio_features,spotify:track:3ibKnFDaa3GhpPGlOUj7ff,0.556,20s0P9QLxGqKuCsGwFsp7w
3,0.40600,https://api.spotify.com/v1/audio-analysis/5knu...,0.720,251088.0,0.763,5knuzwU65gJK7IF5yJsuaW,0.000000,9.0,0.1800,-4.068,0.0,0.0523,101.965,4.0,https://api.spotify.com/v1/tracks/5knuzwU65gJK...,audio_features,spotify:track:5knuzwU65gJK7IF5yJsuaW,0.742,6MDME20pz9RveH9rEXvrOM
4,0.00776,https://api.spotify.com/v1/audio-analysis/1zi7...,0.792,173987.0,0.625,1zi7xx7UVEFkmKfv06H8x0,0.001800,1.0,0.3290,-5.609,1.0,0.0536,103.967,4.0,https://api.spotify.com/v1/tracks/1zi7xx7UVEFk...,audio_features,spotify:track:1zi7xx7UVEFkmKfv06H8x0,0.370,3TVXtAsR1Inumwj472S9r4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.23300,https://api.spotify.com/v1/audio-analysis/6eT7...,0.675,256533.0,0.562,6eT7xZZlB2mwyzJ2sUKG6w,0.000000,7.0,0.0816,-7.678,1.0,0.0352,130.098,4.0,https://api.spotify.com/v1/tracks/6eT7xZZlB2mw...,audio_features,spotify:track:6eT7xZZlB2mwyzJ2sUKG6w,0.309,246dkjvS1zLTtiykXe5h60
2996,0.40700,https://api.spotify.com/v1/audio-analysis/1ISs...,0.493,198253.0,0.738,1ISsiC4Fw6f96kZQegLGiJ,0.000000,5.0,0.1300,-6.921,1.0,0.2620,176.071,4.0,https://api.spotify.com/v1/tracks/1ISsiC4Fw6f9...,audio_features,spotify:track:1ISsiC4Fw6f96kZQegLGiJ,0.219,6fOMl44jA4Sp5b9PpYCkzz
2997,0.03800,https://api.spotify.com/v1/audio-analysis/1h2x...,0.633,245493.0,0.634,1h2xVEoJORqrg71HocgqXd,0.006400,8.0,0.0385,-12.115,1.0,0.0725,100.499,4.0,https://api.spotify.com/v1/tracks/1h2xVEoJORqr...,audio_features,spotify:track:1h2xVEoJORqrg71HocgqXd,0.872,7guDJrEfX3qb6FEbdPA5qi
2998,0.07170,https://api.spotify.com/v1/audio-analysis/1NhP...,0.516,224693.0,0.764,1NhPKVLsHhFUHIOZ32QnS2,0.000000,2.0,0.1150,-6.223,1.0,0.0366,148.021,4.0,https://api.spotify.com/v1/tracks/1NhPKVLsHhFU...,audio_features,spotify:track:1NhPKVLsHhFUHIOZ32QnS2,0.376,5Pwc4xIPtQLFEnJriah9YJ


In [79]:
audioFeaturesDf
df['duration_ms'] = audioFeaturesDf['duration_ms']
df['acousticness'] = audioFeaturesDf['acousticness']
df['danceability'] = audioFeaturesDf['danceability']
df['energy'] = audioFeaturesDf['energy']
df['instrumentalness'] = audioFeaturesDf['instrumentalness']
df['liveness'] = audioFeaturesDf['liveness']
df['loudness'] = audioFeaturesDf['loudness']
df['speechiness'] = audioFeaturesDf['speechiness']
df['valence'] = audioFeaturesDf['valence']
df['tempo'] = audioFeaturesDf['tempo']
df

,track_name,artist,streams,url,date,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
1,Starboy,The Weeknd,3135625,https://open.spotify.com/track/5aAx2yezTd8zXrk...,2017-01-01,244960.0,0.41400,0.748,0.524,0.000000,0.1110,-5.599,0.0338,0.661,95.010
2,Closer,The Chainsmokers,3015525,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,2017-01-01,256733.0,0.23500,0.656,0.578,0.000000,0.1180,-8.970,0.0922,0.556,94.514
3,Let Me Love You,DJ Snake,2545384,https://open.spotify.com/track/4pdPtRcBmOSQDlJ...,2017-01-01,251088.0,0.40600,0.720,0.763,0.000000,0.1800,-4.068,0.0523,0.742,101.965
4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,https://open.spotify.com/track/5knuzwU65gJK7IF...,2017-01-01,173987.0,0.00776,0.792,0.625,0.001800,0.3290,-5.609,0.0536,0.370,103.967
5,One Dance,Drake,2259887,https://open.spotify.com/track/1xznGGDReH1oQq0...,2017-01-01,210937.0,0.10500,0.928,0.481,0.000000,0.1760,-9.350,0.2870,0.613,134.007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,You & Me,Marc E. Bassy,348357,https://open.spotify.com/track/4QdovcM8j50QPly...,2017-01-15,198253.0,0.40700,0.493,0.738,0.000000,0.1300,-6.921,0.2620,0.219,176.071
197,Can't Feel My Face,The Weeknd,347804,https://open.spotify.com/track/6RsWqX8zABZLhZy...,2017-01-15,245493.0,0.03800,0.633,0.634,0.006400,0.0385,-12.115,0.0725,0.872,100.499
198,White Iverson,Post Malone,342787,https://open.spotify.com/track/6eT7xZZlB2mwyzJ...,2017-01-15,224693.0,0.07170,0.516,0.764,0.000000,0.1150,-6.223,0.0366,0.376,148.021
199,Te Quiero Pa´Mi,Don Omar,342480,https://open.spotify.com/track/3BY2mafsbsoKGqS...,2017-01-15,219333.0,0.12700,0.789,0.858,0.000000,0.3490,-5.542,0.3110,0.775,140.022


In [80]:
#visualization
#plotting all the new metrics in our dataframe vs streams

In [81]:
from scipy.stats import normaltest
from numpy.random import seed
from numpy.random import randn
alpha = 0.05
#data = df['tempo'].sample(n=10).array
data = []
for i in range(0,100):
    data.append(np.mean(df['streams'].sample(n=100)))
print(data)
plt.hist(data)
plt.xlabel("Estimate")
plt.ylabel("Frequency")

[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]


ValueError: supplied range of [inf, inf] is not finite

In [ ]:
plt.scatter(df['duration_ms'],df['streams'])
plt.xlabel('duration in milliseconds')
plt.ylabel('streams in millions')

In [ ]:
plt.scatter(df['acousticness'],df['streams'])
plt.xlabel('accousticness scale of 0-1')
plt.ylabel('streams in millions')

In [ ]:
plt.scatter(df['danceability'],df['streams'])
plt.xlabel('danceability scale of 0-1')
plt.ylabel('streams in millions')

In [ ]:
plt.scatter(df['energy'],df['streams'])
plt.xlabel('energy scale of 0-1')
plt.ylabel('streams in millions')

In [ ]:
plt.scatter(df['instrumentalness'],df['streams'])
plt.xlabel('instrumentalness scale of 0-1')
plt.ylabel('streams in millions')

In [ ]:
plt.scatter(df['liveness'],df['streams'])
plt.xlabel('liveness scale of 0-1')
plt.ylabel('streams in millions')

In [ ]:
plt.scatter(df['loudness'],df['streams'])
plt.xlabel('loudness SCALE???????????????????')
plt.ylabel('streams in millions')

In [ ]:
plt.scatter(df['speechiness'],df['streams'])
plt.xlabel('speechiness scale of 0-.5')
plt.ylabel('streams in millions')

In [ ]:
plt.scatter(df['valence'],df['streams'])
plt.xlabel('valence scale of 0-1')
plt.ylabel('streams in millions')

In [ ]:
plt.scatter(df['tempo'],df['streams'])
plt.xlabel('tempo scale of 0-200')
plt.ylabel('streams in millions')

In [ ]:
#violin plot of genre vs streams in millions
ax = sns.violinplot(x='genre', y='streams', data=df, palette='muted')


In [ ]:
plt.scatter(df['date'],df['streams'])
plt.xlabel('date')
plt.ylabel('streams in millions')

Can we find any trends within these features for the first 10 songs on our dataframe (a.k.a. the top 10 songs on the first of January)?

In [ ]:
sliceOfData = df.head(10)